# Latitude-dependent grey radiation

Here is a quick example of using the `climlab.GreyRadiationModel` with a latitude dimension and seasonally varying insolation.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import climlab

In [2]:
model = climlab.GreyRadiationModel(num_lev=30, num_lat=90)
print model

climlab Process of type <class 'climlab.model.column.GreyRadiationModel'>. 
State variables and domain shapes: 
  Tatm: (90, 30) 
  Ts: (90, 1) 
The subprocess tree: 
top: <class 'climlab.model.column.GreyRadiationModel'>
   LW: <class 'climlab.radiation.radiation.Radiation'>
   SW: <class 'climlab.radiation.radiation.RadiationSW'>
   surface: <class 'climlab.surface.surface_radiation.SurfaceRadiation'>
   insolation: <class 'climlab.radiation.insolation.FixedInsolation'>



In [3]:
print model.lat

[-89. -87. -85. -83. -81. -79. -77. -75. -73. -71. -69. -67. -65. -63. -61.
 -59. -57. -55. -53. -51. -49. -47. -45. -43. -41. -39. -37. -35. -33. -31.
 -29. -27. -25. -23. -21. -19. -17. -15. -13. -11.  -9.  -7.  -5.  -3.  -1.
   1.   3.   5.   7.   9.  11.  13.  15.  17.  19.  21.  23.  25.  27.  29.
  31.  33.  35.  37.  39.  41.  43.  45.  47.  49.  51.  53.  55.  57.  59.
  61.  63.  65.  67.  69.  71.  73.  75.  77.  79.  81.  83.  85.  87.  89.]


In [4]:
insolation = climlab.radiation.insolation.DailyInsolation(domains=model.Ts.domain)

In [5]:
model.add_subprocess('insolation', insolation)

In [6]:
print model

climlab Process of type <class 'climlab.model.column.GreyRadiationModel'>. 
State variables and domain shapes: 
  Tatm: (90, 30) 
  Ts: (90, 1) 
The subprocess tree: 
top: <class 'climlab.model.column.GreyRadiationModel'>
   LW: <class 'climlab.radiation.radiation.Radiation'>
   SW: <class 'climlab.radiation.radiation.RadiationSW'>
   surface: <class 'climlab.surface.surface_radiation.SurfaceRadiation'>
   insolation: <class 'climlab.radiation.insolation.DailyInsolation'>



In [7]:
model.step_forward()

ValueError: matrix_multiply: Operand 1 has a mismatch in its core dimension 0, with gufunc signature (m,n),(n,p)->(m,p) (size 120 is different from 31)

In [ ]:
plt.plot(model.lat, model.diagnostics['insolation'])

In [ ]:
model.Tatm.shape

In [ ]:
model.integrate_years(1)

In [ ]:
plt.plot(model.lat, model.Ts)

In [ ]:
model.integrate_years(1)

In [ ]:
plt.plot(model.lat, model.timeave['Ts'])

In [ ]:
plt.contour(model.lat, model.lev, model.timeave['Tatm'].transpose())

In [ ]:
model2 = climlab.RadiativeConvectiveModel(num_lev=30, num_lat=90)
insolation = climlab.radiation.insolation.DailyInsolation(domains=model.Ts.domain)
model2.add_subprocess('insolation', insolation)

In [ ]:
model2.step_forward()

In [ ]:
model2.integrate_years(1)

In [ ]:
plt.contour(model2.lat, model2.lev, model2.Tatm.transpose())

## Testing out multi-dimensional Band Models

In [ ]:
import climlab
import numpy as np

#  Put in some ozone
import netCDF4 as nc

datapath = "http://ramadda.atmos.albany.edu:8080/repository/opendap/latest/Top/Users/Brian+Rose/CESM+runs/"
endstr = "/entry.das"

topo = nc.Dataset( datapath + 'som_input/USGS-gtopo30_1.9x2.5_remap_c050602.nc' + endstr )
ozone = nc.Dataset( datapath + 'som_input/ozone_1.9x2.5_L26_2000clim_c091112.nc' + endstr )

#  Dimensions of the ozone file
lat = ozone.variables['lat'][:]
lon = ozone.variables['lon'][:]
lev = ozone.variables['lev'][:]

# Taking annual, zonal average of the ozone data
O3_zon = np.mean( ozone.variables['O3'],axis=(0,3) )

In [ ]:
import climlab
import numpy as np
#  make a model on the same grid as the ozone
model3 = climlab.BandRCModel(lev=lev, lat=lat)
insolation = climlab.radiation.insolation.DailyInsolation(domains=model3.Ts.domain)
model3.add_subprocess('insolation', insolation)
print model3

In [ ]:
#  Set the ozone mixing ratio
#   IMPORTANT: we need to flip the ozone array around because the vertical coordinate runs the wrong way
#   (first element is top of atmosphere, whereas our model expects the first element to be just above the surface)
O3_trans = np.transpose(O3_zon)
print O3_trans.shape
print lev
print model3.lev


In [ ]:
print O3_trans[0,:]
O3_flipped = np.fliplr(O3_trans)
print O3_flipped[0,:]

In [ ]:
model3.absorber_vmr['O3'] = O3_flipped

In [ ]:
print model3.absorber_vmr['O3'].shape
print model3.Tatm.shape

In [ ]:
model3.integrate_years(1.)

In [ ]:
plt.contour(model3.lat, model3.lev, model3.Tatm.transpose())

This runs but doesn't seem to be doing the right calculation.